In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from irt.evaluation import eval_utils
from irt.data import data_loader
import matplotlib.pyplot as plt
import time
import girth
# from girth import rasch_conditional
from irt.data.rasch import generate_data
from irt.algorithms import spectral_estimator
from irt.algorithms import conditional_mle
from irt.algorithms import rasch_mml

In [3]:
# A = data_loader.lsat()
A = data_loader.ml_100k()

In [ ]:
n_trials = 10

auc_cmle = []
auc_mmle = []
auc_ase = []

for i in range(n_trials):
    data, test = eval_utils.partition_data(A)
    # try:
    #     auc_cmle.append(eval_utils.evaluate_auc(conditional_mle.rasch_conditional, data, test))
    # except Exception as e:
    #     auc_cmle.append(np.nan)
    #     continue
        
    try:
        auc_mmle.append(eval_utils.evaluate_auc(rasch_mml.rasch_mml, data, test))
    except Exception as e:
        auc_mmle.append(np.nan)
        continue
    
    try:
        auc_ase.append(eval_utils.evaluate_auc(spectral_estimator.spectral_estimate, data, test))
    except Exception as e:
        auc_ase.append(np.nan)
        continue


In [ ]:
print("CMLE", np.nanmean(auc_cmle))
print("MMLE", np.nanmean(auc_mmle))
print("ASE", np.nanmean(auc_ase))
